In [1]:
from keras import backend as K
import tensorflow as tf
from math import ceil
import numpy as np
import math
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy32
from bounding_box_utils.bounding_box_utils_tf import tf_iou
from bounding_box_utils.bounding_box_utils import convert_coordinates, iou_float
import timeit

Using TensorFlow backend.


In [2]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
K.set_session(sess)
K.clear_session()


In [3]:
class CodeTimer:
    def __init__(self, name=None):
        self.name = " '"  + name + "'" if name else ''

    def __enter__(self):
        self.start = timeit.default_timer()

    def __exit__(self, exc_type, exc_value, traceback):
        self.took = (timeit.default_timer() - self.start) * 1000.0
        print('Code block' + self.name + ' took: ' + str(self.took) + ' ms')


In [4]:
gt = np.load("outputs/predder.npy")
gt1 = gt[1]['predictions_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_to_2']
gt2_proj = gt[1]['predictions_2_to_1']


In [5]:
pred_1 = np.load("outputs/predictions_1_e0.npy")
pred_1_to_2 = np.load("outputs/predictions_1_to_2_e0.npy")
pred_2 = np.load("outputs/predictions_2_e0.npy")
pred_2_proj = np.load("outputs/predictions_2_to_1_e0.npy")


In [6]:
neg_pos_ratio = 3
n_neg_min = 0
alpha = 1

def smooth_L1_loss(y_true, y_pred):
    absolute_loss = tf.abs(y_true - y_pred)
    square_loss = 0.5 * (y_true - y_pred)**2
    l1_loss = tf.where(tf.less(absolute_loss, 1.0), square_loss, absolute_loss - 0.5)
    return tf.reduce_sum(l1_loss, axis=-1)

def log_loss(y_true, y_pred):

    y_pred = tf.maximum(y_pred, 1e-15)
    # Compute the log loss
    log_loss = -tf.reduce_sum(y_true * tf.log(y_pred), axis=-1)
    return log_loss


def compute_loss(y_true, y_pred):

    def gt_rem(pred, gt):
        with CodeTimer('gt_rem'):
        # predval = tf.shape(pred)
        # gtval = tf.shape(gt)
            val = tf.subtract(tf.shape(pred)[1],tf.shape(gt)[1])
            gt = tf.slice(gt, [0, 0, 0], [1, tf.shape(pred)[0], 18],name="rem_slice")
        return gt

    def gt_add(pred, gt):
        with CodeTimer('gt_add'):
            a = tf.shape(pred)[1]
            b = tf.shape(gt)[1]

            val = tf.subtract(tf.shape(pred),tf.shape(gt))
            val = tf.cast(val[1], tf.int32)
            ext = tf.slice(gt, [0, 0, 0], [1, 1, 18], name="add_slice")
            multiply = [1,val,1]
            ext = tf.tile(ext, multiply)
            gt = K.concatenate([ext,gt], axis=1)
        return gt

    def equalalready(gt, pred): return pred

    def make_equal(pred, gt):
        equal_tensor = tf.cond(tf.shape(pred)[1] < tf.shape(gt)[1], lambda: gt_rem(pred, gt), lambda: gt_add(pred, gt), name="make_equal_cond")
        return equal_tensor


    def matcher(y_true_1,y_pred_1,y_true_2,y_pred_2, bsz):
        pred = 0
        gt = 0

        for i in range(bsz):
            print("i: ",i)
            with CodeTimer('filtering'):
                filterer = tf.where(tf.not_equal(y_true_1[i,:,-4],99))
                filterer_2 = tf.where(tf.not_equal(y_true_2[i,:,-4],99))
                
                y_true_new = tf.gather_nd(y_true_1[i,:,:],filterer)            
                y_true_new = tf.expand_dims(y_true_new, 0)
                y_true_2_new = tf.gather_nd(y_true_2[i,:,:],filterer_2)
                y_true_2_new = tf.expand_dims(y_true_2_new, 0)
                
                set1 = tf.cast(y_true_new[:,:,-4],dtype=tf.int32)
                set2 = tf.cast(y_true_2_new[:,:,-4],dtype=tf.int32)
                
                id_pick = tf.sets.set_intersection(set1[None,:], set2[None, :])
                id_pick = tf.cast(id_pick.values[0],dtype=tf.float32)
                
                filterer = tf.where(tf.equal(y_true_1[i,:,-4],id_pick))
                filterer_2 = tf.where(tf.equal(y_true_2[i,:,-4],id_pick))
                
                y_true_new = tf.gather_nd(y_true_1[i,:,:],filterer) 
                y_true_new = tf.expand_dims(y_true_new, 0)
                
                y_true_2_new = tf.gather_nd(y_true_2[i,:,:],filterer_2)
                y_true_2_new = tf.expand_dims(y_true_2_new, 0)
                y_pred_1_new = y_pred_1[i,:,-16:-12]
                
            with CodeTimer('iou_out'):   
#                 iou_out = tf.py_func(iou_float, [y_pred_1_new,y_true_new[0,:,-16:-12]], tf.float32, name="iou_out")
                iou_out = tf_iou(y_pred_1_new,y_true_new[0,:,-16:-12])
    
                print(K.eval(iou_out).shape)
            with CodeTimer('bipartite_matches'):
                bipartite_matches = tf.py_func(match_bipartite_greedy32, [iou_out], tf.int32, name="bipartite_matches")
                print(K.eval(bipartite_matches).shape)
#             print("bipartite_matches ",K.eval(bipartite_matches).shape)
#             print("y_pred_2[i,:,:] ",K.eval(y_pred_2[i,:,:]).shape)

            with CodeTimer('out'):
        
                out = tf.gather(y_pred_2[i,:,:], [bipartite_matches], axis=0, name="out")
                print(K.eval(out).shape)
            with CodeTimer('box_comparer'):
                
                box_comparer = tf.reduce_all(tf.equal(tf.shape(out)[1], tf.shape(y_true_2_new)[1]), name="box_comparer")
                print(K.eval(box_comparer).shape)
            with CodeTimer('y_true_2_equal'):
                y_true_2_equal = tf.cond(box_comparer, lambda: equalalready(out, y_true_2_new), lambda: make_equal(out, y_true_2_new), name="y_true_cond")
                print(K.eval(y_true_2_equal).shape)
            with CodeTimer('concatentation'):

                if i != 0:
                    pred = K.concatenate([pred,out], axis=-1)
                    gt = K.concatenate([gt,y_true_2_equal], axis=0)
                else:
                    pred = out
                    gt = y_true_2_equal    
        return pred, gt

        
    y_true_1 = y_true[:,:,:18]
    y_pred_1 = y_pred[:,:,:18]
    y_true_2 = y_true[:,:,18:]
    y_pred_2 = y_pred[:,:,18:]

    with CodeTimer('matcher'):
        y_pred_out, y_true_out = matcher(y_true_1,y_pred_1,y_true_2,y_pred_2,1)
        print("y_pred: ",K.eval(y_pred_out).shape)
        print("y_true: ",K.eval(y_true_out).shape)
    
#     y_pred = tf.cast(y_pred, dtype=tf.float32)
#     y_true = tf.cast(y_true, dtype=tf.float32)
    
    with CodeTimer('box'):

        batch_size = tf.shape(y_pred_out)[0]
        n_boxes = tf.shape(y_true_out)[1] 

    with CodeTimer('classification_loss'):
        classification_loss = tf.to_float(log_loss(y_true_out[:,:,:-16], y_pred_out[:,:,:-16])) # Output shape: (batch_size, n_boxes)
    with CodeTimer('localization_loss'):
        localization_loss = tf.to_float(smooth_L1_loss(y_true_out[:,:,-16:-12], y_pred_out[:,:,-16:-12])) # Output shape: (batch_size, n_boxes)
    with CodeTimer('negatives'):
        negatives = y_true_out[:,:,0] # Tensor of shape (batch_size, n_boxes)
        positives = tf.to_float(tf.reduce_max(y_true_out[:,:,1:-16], axis=-1)) # Tensor of shape (batch_size, n_boxes)
        n_positive = tf.reduce_sum(positives)
    with CodeTimer('pos_class_loss'):
        pos_class_loss = tf.reduce_sum(classification_loss * positives, axis=-1) # Tensor of shape (batch_size,)

    with CodeTimer('neg_class_loss_all'):

        neg_class_loss_all = classification_loss * negatives # Tensor of shape (batch_size, n_boxes)
        n_neg_losses = tf.count_nonzero(neg_class_loss_all, dtype=tf.int32) # The number of non-zero loss entries in `neg_class_loss_all`
        n_negative_keep = tf.minimum(tf.maximum(neg_pos_ratio * tf.to_int32(n_positive), n_neg_min), n_neg_losses)

    def f1():
        return tf.zeros([batch_size])
    def f2():

        neg_class_loss_all_1D = tf.reshape(neg_class_loss_all, [-1]) # Tensor of shape (batch_size * n_boxes,)
        values, indices = tf.nn.top_k(neg_class_loss_all_1D,
                                      k=n_negative_keep,
                                      sorted=False) # We don't need them sorted.

        negatives_keep = tf.scatter_nd(indices=tf.expand_dims(indices, axis=1),
                                       updates=tf.ones_like(indices, dtype=tf.int32),
                                       shape=tf.shape(neg_class_loss_all_1D)) # Tensor of shape (batch_size * n_boxes,)
        negatives_keep = tf.to_float(tf.reshape(negatives_keep, [batch_size, n_boxes])) # Tensor of shape (batch_size, n_boxes)
        # ...and use it to keep only those boxes and mask all other classification losses
        neg_class_loss = tf.reduce_sum(classification_loss * negatives_keep, axis=-1) # Tensor of shape (batch_size,)
        return neg_class_loss
    
    with CodeTimer('neg_class_loss_all'):
        neg_class_loss = tf.cond(tf.equal(n_neg_losses, tf.constant(0)), f1, f2)

        class_loss = pos_class_loss + neg_class_loss # Tensor of shape (batch_size,)

        loc_loss = tf.reduce_sum(localization_loss * positives, axis=-1) # Tensor of shape (batch_size,)

    # 4: Compute the total loss.
    with CodeTimer('total_loss'):
        total_loss = (class_loss + alpha * loc_loss) / tf.maximum(1.0, n_positive) # In case `n_positive == 0`
        total_loss = total_loss * tf.to_float(batch_size)
        total_loss.set_shape((None,))
        print(K.eval(total_loss))
    return total_loss


In [7]:
print(gt1_proj.shape)
gt1_proj = tf.convert_to_tensor(gt1_proj,dtype=np.float32)
pred_1_to_2 = tf.convert_to_tensor(pred_1_to_2,dtype=np.float32)
loss = compute_loss(gt1_proj,pred_1_to_2)
print("loss:",K.eval(loss))

(4, 17292, 36)
i:  0
Code block 'filtering' took: 60.20039296709001 ms
(17292, 4)
Code block 'iou_out' took: 336.7081810720265 ms
(17292,)
Code block 'bipartite_matches' took: 21166.918084025383 ms
(1, 17292, 18)
Code block 'out' took: 20915.782656986266 ms
()
Code block 'box_comparer' took: 20904.535137000494 ms
Code block 'gt_rem' took: 22.09624508395791 ms
Code block 'gt_add' took: 137.30286492500454 ms
(1, 17292, 18)
Code block 'y_true_2_equal' took: 21047.589571913704 ms
Code block 'concatentation' took: 0.007123919203877449 ms
y_pred:  (1, 17292, 18)
y_true:  (1, 17292, 18)
Code block 'matcher' took: 125875.17393392045 ms
Code block 'box' took: 12.275652959942818 ms
Code block 'classification_loss' took: 11.934897978790104 ms
Code block 'localization_loss' took: 14.568821992725134 ms
Code block 'negatives' took: 9.315431001596153 ms
Code block 'pos_class_loss' took: 2.085180953145027 ms
Code block 'neg_class_loss_all' took: 9.473062003962696 ms
Code block 'neg_class_loss_all' too